## Simple GenAI App using LangChain

In [46]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [106]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader("https://medium.com/@manan.kher22/the-highs-and-lows-of-building-a-computer-engineering-project-12ba7a07bb36")
loader

In [107]:
docs = loader.load()
docs[0].page_content

'The Highs and Lows of Building a Computer Engineering Project | by Manan Kher | MediumOpen in appSign upSign inWriteSign upSign inThe Highs and Lows of Building a Computer Engineering ProjectManan Kher·Follow8 min read·May 15, 2024--1ListenShareHave you ever built a project for your computer engineering major? If yes, how was your experience? It must have likely been a thrilling journey, filled with emotions of joy, excitement and satisfaction, as well as pain and regret, and most importantly, those golden ‘eureka moments’ or the ‘light bulb moments’. My name is Manan Kher, a sophomore doing my major in Computer Engineering, and I was recently blessed with the opportunity to build my first full-fledged project during our 4th semester. I am delighted to share my journey with you, and I would like to highlight the learnings that I have obtained from the mistakes and victories (mostly mistakes) while making the project.A few weeks after the start of 4th semester, we received an email fro

In [108]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=80)
text_splitter

In [109]:
final_docs = text_splitter.split_documents(docs)
final_docs

[Document(metadata={'source': 'https://medium.com/@manan.kher22/the-highs-and-lows-of-building-a-computer-engineering-project-12ba7a07bb36', 'title': 'The Highs and Lows of Building a Computer Engineering Project | by Manan Kher | Medium', 'description': 'Have you ever built a project for your computer engineering major? If yes, how was your experience? It must have likely been a thrilling journey, filled with emotions of joy, excitement and…', 'language': 'en'}, page_content='The Highs and Lows of Building a Computer Engineering Project | by Manan Kher | MediumOpen in appSign upSign inWriteSign upSign inThe Highs and Lows of Building a Computer Engineering ProjectManan Kher·Follow8 min read·May 15, 2024--1ListenShareHave you ever built a project for your computer engineering major? If yes, how was your experience? It must have likely been a thrilling journey, filled'),
 Document(metadata={'source': 'https://medium.com/@manan.kher22/the-highs-and-lows-of-building-a-computer-engineering

In [110]:
len(final_docs)

34

In [111]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='llama3', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [112]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(final_docs, embeddings)

In [113]:
db

In [124]:
query = " While brainstorming ideas, Atharva came up with a fresh and highly advanced suggestion. He suggested that we could implement the Bayesian Search Theory for a practical purpose such as searching for a lost object in a water body."
result = db.similarity_search_with_score(query)

In [125]:
result

[(Document(metadata={'source': 'https://medium.com/@manan.kher22/the-highs-and-lows-of-building-a-computer-engineering-project-12ba7a07bb36', 'title': 'The Highs and Lows of Building a Computer Engineering Project | by Manan Kher | Medium', 'description': 'Have you ever built a project for your computer engineering major? If yes, how was your experience? It must have likely been a thrilling journey, filled with emotions of joy, excitement and…', 'language': 'en'}, page_content='the map and ensuring that the markers and flight path were accurately displayed. Debugging the map integration took considerable time, but her persistence paid off.Our website’s home pageThe Service PageAfter entering the relevant dataAtharva focused on the automation and core algorithms. The website uses Selenium in the background to automate downloading ocean depth data from the GEBCO website.'),
  16351.34),
 (Document(metadata={'source': 'https://medium.com/@manan.kher22/the-highs-and-lows-of-building-a-comp

In [126]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")
llm

Ollama(model='llama3')

In [86]:
llm.invoke("First man to land on moon")

'A historic moment!\n\nThe first person to set foot on the Moon was Neil Armstrong, an American astronaut and commander of the Apollo 11 mission. On July 20, 1969, Armstrong stepped out of the lunar module Eagle and onto the Moon\'s surface at 2:56 UTC (Coordinated Universal Time). He famously declared, "That\'s one small step for man, one giant leap for mankind" as he took his first steps.\n\nArmstrong was joined on the mission by fellow astronauts Edwin "Buzz" Aldrin and Michael Collins. The Apollo 11 spacecraft carried the three astronauts into lunar orbit, where they prepared to land on the Moon\'s surface.\n\nThe successful landing marked a historic achievement in space exploration, marking the first time humans had visited another celestial body. Armstrong\'s footprints remained on the Moon\'s surface for over 50 years until NASA\'s Lunar Reconnaissance Orbiter captured new images of the site in 2019.\n\nArmstrong\'s remarkable feat will always be remembered as a pivotal moment i

In [132]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the question asked by the user based only on the provided context:
<context>
{context}
</context>

"""
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the question asked by the user based only on the provided context:\n<context>\n{context}\n</context>\n\n'))])
| Ollama(model='llama3')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [133]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002046D86BFD0>)

In [135]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002046D86BFD0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the question asked by the user based only on the provided context:\n<context>\n{context}\n</context>\n\n'))])
            | Ollama(model='llama3')
            | StrOutputParser(), config={'run_name': 'stuff_documents_chain'})
  }), config={'run_name': 'retrieval_chain'})

In [137]:
retrieval_chain.invoke({"input":"What is the title of the project?"})

{'input': 'What is the title of the project?',
 'context': [Document(metadata={'source': 'https://medium.com/@manan.kher22/the-highs-and-lows-of-building-a-computer-engineering-project-12ba7a07bb36', 'title': 'The Highs and Lows of Building a Computer Engineering Project | by Manan Kher | Medium', 'description': 'Have you ever built a project for your computer engineering major? If yes, how was your experience? It must have likely been a thrilling journey, filled with emotions of joy, excitement and…', 'language': 'en'}, page_content='the map and ensuring that the markers and flight path were accurately displayed. Debugging the map integration took considerable time, but her persistence paid off.Our website’s home pageThe Service PageAfter entering the relevant dataAtharva focused on the automation and core algorithms. The website uses Selenium in the background to automate downloading ocean depth data from the GEBCO website.'),
  Document(metadata={'source': 'https://medium.com/@manan

In [147]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = retriever|prompt|llm|output_parser

chain

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002046D86BFD0>)
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the question asked by the user based only on the provided context:\n<context>\n{context}\n</context>\n\n'))])
| Ollama(model='llama3')
| StrOutputParser()

In [148]:
chain.invoke({"input":"What is the title of the project?"})

'The question being asked based on the provided context is:\n\n*None*\n\nThere is no specific question asked in the given context. However, some possible questions that can be inferred from the text are:\n\n* How was your experience building a computer engineering project?\n* What were the highs and lows of building a computer engineering project?'